<a href="https://colab.research.google.com/github/Adisriman/AlexandriaRepo/blob/master/imdb_movie_review_classification_tf_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub


TensorFlow 2.x selected.


In [3]:
tf.__version__ #check the tensorflow version

'2.1.0'

In [5]:
!pip install tensorflow - -upgrade 


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [6]:
tfds.list_builders() # check for the available datasets inside tensor_flow

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'groove',
 'higgs',
 'horses_or_humans',
 'i_natura

In [0]:
(train_data,validation_data,test_data),metadata = tfds.load(
    'imdb_reviews',split=['train[:80%]','train[80%:90%]','train[20%:]'],
    with_info=True,
    as_supervised=True,
)#create 3 data sets,for training,testing and validation - training-80% from 

In [11]:
train_data

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [14]:
train_examples_batch,train_labels_batch =next(iter(train_data.batch(10))) #read 10 use-reviews.
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [15]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [0]:
pretrained_model="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"               #convert into numerical value (texts)
hub_layer=hub.KerasLayer(pretrained_model,input_shape=[],dtype=tf.string,trainable=True)

In [18]:
hub_layer(train_examples_batch[:2]) #encode view of the first two review feedback

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

In [19]:
model=tf.keras.Sequential()
model.add(hub_layer) #input layer is the encoded output from tensorflow hub nural network
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) # back_propagation configuration

In [24]:
model.fit(train_data.shuffle(10000).batch(512),epochs=20,validation_data=validation_data.batch(512),verbose=1) #get ready maximum buffer or 100000 record for training purpose and shuffle 
                                   # suffle because the increase the randomness and avoid the overfitting , 512 is for ram performance                                #usually, batch sizes are defined as series of 32,64,128,256,512,...more it become big,

Epoch 1/20
40/40 [==============================] - 6s 139ms/step - loss: 1.1239 - accuracy: 0.5137 - val_loss: 0.7036 - val_accuracy: 0.5500
Epoch 2/20
40/40 [==============================] - 5s 125ms/step - loss: 0.6758 - accuracy: 0.5864 - val_loss: 0.6374 - val_accuracy: 0.6316
Epoch 3/20
40/40 [==============================] - 5s 127ms/step - loss: 0.6221 - accuracy: 0.6618 - val_loss: 0.5908 - val_accuracy: 0.6908
Epoch 4/20
40/40 [==============================] - 5s 131ms/step - loss: 0.5688 - accuracy: 0.7187 - val_loss: 0.5391 - val_accuracy: 0.7372
Epoch 5/20
40/40 [==============================] - 5s 131ms/step - loss: 0.5119 - accuracy: 0.7696 - val_loss: 0.4895 - val_accuracy: 0.7844
Epoch 6/20
40/40 [==============================] - 5s 129ms/step - loss: 0.4609 - accuracy: 0.8029 - val_loss: 0.4466 - val_accuracy: 0.8060
Epoch 7/20
40/40 [==============================] - 5s 130ms/step - loss: 0.4151 - accuracy: 0.8271 - val_loss: 0.4094 - val_accuracy: 0.8292
Epoch 

In [25]:
model.predict(["This is the worst movie i have ever seen","an excellent moview i ever enjoyed"])

array([[0.14935918],
       [0.99656004]], dtype=float32)

In [26]:
model.predict(['fuck off'])   #transfer learning

array([[0.21635582]], dtype=float32)